In [1]:
from tools.tools import *
import pickle
from pathlib import Path
torch.manual_seed(1)
import optuna


In [2]:
# Load the pickled data
with open(Path("data/data.bin"), "rb") as f:
    data = pickle.load(f)
data = flatten_dict(data) # flatten the dictionary
data, min_max_dict = scale_dict(data) # scale the dictionary
data_lad, data_ela = format_LGHE4C25B01(min_max_dict,constant_temperature=True) # load and format the charge-discharge data
datasets = {'lad': data_lad, 'ela': data_ela} # dictionary containing the above defined charge-discharge data

In [3]:
# Hyperparametertuning with Optuna
def objective(trial, datasets):
    # Define hyperparameters for this trial
    sequence_length = trial.suggest_int("sequence_length", 1, 5)
    batch_size = trial.suggest_categorical("batch_size", [8,16,32,64,128])
    output_keys = ["V"]
    input_keys = ['soc','I','T']
    input_size = len(input_keys)
    print(f"The input keys are: {input_keys} with size {input_size}")
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    epochs = trial.suggest_int("epochs", 10, 50)
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)


    # Create and train the model
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

    model = LSTM_model(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=1)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    print(f"Trial parameters\n{trial.params}")

    for i in range(0,len(data),6):
        sub_dict = get_sub_dictionary(data, i)
        dataset = LSTM_dataset(sub_dict, sequence_length=sequence_length, input_keys=input_keys, output_keys=output_keys) 
        train_loop(model=model,
                    batch_size=batch_size,
                    dataset=dataset,
                    num_epochs=epochs,
                    optimizer=optimizer,
                    device=device,
                    print_perf=False)

    # Evaluate the model on both testing sets
    
    total_loss = 0
    for set_name, dataset in datasets.items():
        for key, value in dataset.items():
            real_dataset = LSTM_dataset(value, sequence_length=sequence_length)
            test_loss, output_prediction, output_real = test_model(model, batch_size=batch_size, test_set=real_dataset, device=device, return_prediction=True)
            total_loss += test_loss
            print(f"At temperature {key} the loss is {test_loss}")
    avg_loss = total_loss / (len(data_lad) + len(data_ela))

    return avg_loss, model, trial.params # Return the objective value

In [4]:
# Run objective function
n_trials = 100  # number of trials to train the lstm
study = optuna.create_study(direction='minimize')  # the purpose is to minimize the loss

# Define variables to store the best model, training loss value and best parameters
best_model = None
best_value = float('inf')
best_params = None

# If no improvement in consecutively 10 trials then drop the training
no_improvement_trials = 0
max_no_improvement_trials = 10

# Loop over the trials 
for i in range(n_trials):
    trial = study.ask()
    value, model, params = objective(trial, datasets=datasets)
    study.tell(trial, value)
    if value < best_value:
        best_value = value
        best_model = model
        best_params= params
        no_improvement_trials = 0
    else:
        no_improvement_trials += 1
        print(f"Number of no improvement trials is {no_improvement_trials}")
    if no_improvement_trials >= max_no_improvement_trials:
        print(f"No improvement in the last {no_improvement_trials} trials. Stopping...")
        break
    print(f'Trial {i+1}/{n_trials} completed. Remaining trials: {n_trials - i - 1}')

# Print the found parameters
print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2023-07-11 02:01:10,837] A new study created in memory with name: no-name-99620cea-0e9c-4fad-8398-7b0f13fecde1


The input keys are: ['soc', 'I', 'T'] with size 3
Trial parameters
{'sequence_length': 5, 'batch_size': 64, 'hidden_size': 210, 'num_layers': 5, 'epochs': 30, 'lr': 0.0001445972491807313}
The current Dataset's key is: 15_FC_0.5_
The current Dataset's key is: 15_FC_1_
The current Dataset's key is: 15_FC_2_
The current Dataset's key is: 15_PC_0.1_0.2_0.5_
The current Dataset's key is: 15_PC_0.1_0.2_1_
The current Dataset's key is: 15_PC_0.1_0.2_2_
The current Dataset's key is: 15_PC_0.2_0.2_0.5_
The current Dataset's key is: 15_PC_0.2_0.2_1_
The current Dataset's key is: 15_PC_0.2_0.2_2_
The current Dataset's key is: 15_PC_0.2_0.4_0.5_
The current Dataset's key is: 15_PC_0.2_0.4_1_
The current Dataset's key is: 15_PC_0.2_0.4_2_
The current Dataset's key is: 15_PC_0.3_0.2_0.5_
The current Dataset's key is: 15_PC_0.3_0.2_1_
The current Dataset's key is: 15_PC_0.3_0.2_2_
The current Dataset's key is: 15_PC_0.3_0.4_0.5_
The current Dataset's key is: 15_PC_0.3_0.4_1_
The current Dataset's key

In [ ]:
# Save the best model
torch.save(best_model.state_dict(), 'best_lstm_ecm_trained.pth')